In [1]:
raw_data_path = "../data/20min-test-query-2020-jan-jun/raw-data/"
raw_file_name = "cd558cca-53cb-4ff9-a3f5-89f70e139051__2024_10_13T18_23_54.tsv"
intermediate_data_path = "../data/20min-test-query-2020-jan-jun/intermediate-data/"

In [2]:
import pandas as pd
import os
from pathlib import Path

In [3]:
with open(os.path.join(raw_data_path, raw_file_name)) as file:
    raw_file = file.readlines()

print(raw_file[0])
print(raw_file[1].replace("\t", " <tab> "))

id	pubtime	medium_code	medium_name	rubric	regional	doctype	doctype_description	language	char_count	dateline	head	subhead	article_link	content_id	content

38899273 <tab> 2020-02-10 08:28:00+01 <tab> ZWAO <tab> 20 minuten online <tab>  <tab>  <tab> WWE <tab> Online medium <tab> de <tab> 562 <tab> Singapore Airlines <tab> A380 muss wegen Orkantief durchstarten <tab>  <tab> https://www.20min.ch/schweiz/zuerich/story/24250135 <tab> 00028ac0-46af-cb40-908b-dedebaf3a9ea <tab> <tx><ld><p>Sturmtief Sabine wirbelt den Plan des Flughafen Zürich durcheinander. Ein Video eines Leser-Reporters zeigt den A380, der die Landung abbrechen muss.</p></ld><p>Der A380 der Singapore Airline musste den Landeanflug auf den Flughafen Kloten abbrechen. Beim zweiten Mal hat es geklappt wie ein Video eines Leser-Reporters zeigt.</p><p>Nicht nur der A380-Gigant musste durchstarten. Wie ein Leser-Reporter berichtet, traf es auch eine A330-300 der Swiss. Der Flieger kam von Tel Aviv.<br/><br/></p><p>(fss)</p></tx>



In [4]:
df = pd.read_csv(os.path.join(raw_data_path, raw_file_name), sep="\t")
print(df.shape)
print(df.head(1).transpose())

(23555, 16)
                                                                     0
id                                                            38899273
pubtime                                         2020-02-10 08:28:00+01
medium_code                                                       ZWAO
medium_name                                          20 minuten online
rubric                                                             NaN
regional                                                           NaN
doctype                                                            WWE
doctype_description                                      Online medium
language                                                            de
char_count                                                         562
dateline                                            Singapore Airlines
head                            A380 muss wegen Orkantief durchstarten
subhead                                                          

In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23555 entries, 0 to 23554
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   23555 non-null  int64  
 1   pubtime              23555 non-null  object 
 2   medium_code          23555 non-null  object 
 3   medium_name          23555 non-null  object 
 4   rubric               0 non-null      float64
 5   regional             0 non-null      float64
 6   doctype              23555 non-null  object 
 7   doctype_description  23555 non-null  object 
 8   language             23555 non-null  object 
 9   char_count           23555 non-null  int64  
 10  dateline             23267 non-null  object 
 11  head                 23555 non-null  object 
 12  subhead              0 non-null      float64
 13  article_link         23554 non-null  object 
 14  content_id           23555 non-null  object 
 15  content              23555 non-null 

In [6]:
drop_nan_columns = ["rubric", "regional", "subhead"]
df = df.drop(drop_nan_columns, axis=1)
df.shape

(23555, 13)

In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23555 entries, 0 to 23554
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   23555 non-null  int64 
 1   pubtime              23555 non-null  object
 2   medium_code          23555 non-null  object
 3   medium_name          23555 non-null  object
 4   doctype              23555 non-null  object
 5   doctype_description  23555 non-null  object
 6   language             23555 non-null  object
 7   char_count           23555 non-null  int64 
 8   dateline             23267 non-null  object
 9   head                 23555 non-null  object
 10  article_link         23554 non-null  object
 11  content_id           23555 non-null  object
 12  content              23555 non-null  object
dtypes: int64(2), object(11)
memory usage: 2.3+ MB
None


In [8]:
df['content']

0        <tx><ld><p>Sturmtief Sabine wirbelt den Plan d...
1        <tx><ld><p>Les voisins chanteurs de la rotonde...
2        <tx><ld><p>Der Kanton Aargau führt im Kampf ge...
3        <tx><ld><p>Inmitten der anderen Automobile der...
4        <tx><ld><p>Le ministre de la santé Olivier Vér...
                               ...                        
23550    <tx><ld><p>La collecte de sapins de Noël a été...
23551    <tx><ld><p>Un porte-parole du Département fédé...
23552    <tx><ld><p>L'incendie d'un véhicule s'est prop...
23553    <tx><ld><p>Comme l'épidémie de Covid-19 se pou...
23554    <tx><ld><p>Tandis que certains ont critiqué le...
Name: content, Length: 23555, dtype: object